In [3]:
import lance

r = lance.dataset("/home/maximilien/work/bop_data/bop_ds.lance/")

In [4]:
for batch in r.to_batches(limit=10):
    a = batch.to_pandas()
    break


In [5]:
import duckdb

duckdb.query("SELECT id, category_id FROM a").to_df()

,id,category_id
0,000005_000065,"[1, 2, 3]"
1,000003_000072,[2]
2,000014_000070,"[9, 9, 9, 9, 9, 10, 10, 11, 11, 12, 12]"
3,000006_000141,"[1, 2, 3]"
4,000003_000246,"[1, 2, 2, 3]"
5,000004_000185,"[1, 2, 2, 3]"
6,000006_000230,"[1, 2, 3]"
7,000014_000086,"[9, 9, 9, 9, 9, 10, 10, 11, 12, 12]"
8,000004_000173,"[1, 2, 2, 3]"
9,000012_000136,"[9, 9, 9, 10, 10, 11, 11, 12, 12]"


In [6]:
import json
import pandas as pd

with open('/home/maximilien/work/adapt-2023_with_gt/info.json', 'r') as json_file:
    info = json.load(json_file)

with open('/home/maximilien/work/adapt-2023_with_gt/test_targets_adapt23.json', 'r') as json_file:
    data = json.load(json_file)



df = pd.DataFrame(data)
info_df = pd.DataFrame(info)

# Renommer la colonne "obj_id" en "category_id"
df.rename(columns={'obj_id': 'category_id'}, inplace=True)


result_table = df.groupby(['category_id'])['scene_id'].agg(set).reset_index()

# Renommer la colonne "scene_id" en "scenes_ids"
result_table.rename(columns={'scene_id': 'scenes_ids'}, inplace=True)

# Compter le nombre d'occurrences
result_table['number of occurrences'] = df.groupby(['category_id'])['scene_id'].size().values

result_table['category_id'] = result_table['category_id'].apply(lambda i: info['category_index'][str(i)])

result_table = result_table.set_index(['category_id']).join(info_df.set_index(['category_index']), how='inner').reset_index()

result_table = result_table.get(['index', 'scenes_ids', 'number of occurrences', 'symmetries', 'bbox'])

In [7]:
# Fonction pour calculer le diamètre et le size_ratio à partir des valeurs bbox
def calculate_size(bbox):
    diameter = (bbox[0] ** 2 + bbox[1] ** 2 + bbox[2] **2) ** 0.5
    size_ratio = min(bbox)/max(bbox)
    return diameter, size_ratio

result_table[['diameter', 'size_ratio']] = result_table['bbox'].apply(lambda bbox: pd.Series(calculate_size(bbox)))

result_table.rename(columns={'bbox': 'bbox (mm)'}, inplace=True)

result_table


,index,scenes_ids,number of occurrences,symmetries,bbox (mm),diameter,size_ratio
0,Top_casing,"{3, 4, 5, 6, 15, 17, 18, 19}",1865,yes,"[38, 15, 13]",42.871902,0.342105
1,gear,"{3, 4, 5, 6, 15, 16, 17, 18, 19}",2259,yes,"[25, 25, 12]",37.336309,0.480000
2,Bottom_casing,"{3, 4, 5, 6, 15, 16, 17, 18, 19}",1941,yes,"[38, 15, 14]",43.185646,0.368421
3,carrier,"{7, 8, 9, 10, 15, 16, 17, 18, 19}",2129,yes,"[35, 35, 6]",49.859803,0.171429
4,cover,"{7, 8, 9, 10, 15, 16, 17, 18, 19}",2056,yes,"[42, 42, 8]",59.933296,0.190476
5,housing,"{7, 8, 9, 10, 15, 16, 17, 18, 19}",1909,yes,"[42, 42, 31]",67.000000,0.738095
6,planet,"{7, 8, 9, 10, 15, 16, 17, 18, 19}",2168,yes,"[16, 16, 10]",24.738634,0.625000
7,sun,"{7, 8, 9, 10, 15, 16, 17, 18, 19}",1870,yes,"[20, 20, 22]",35.832946,0.909091
8,Button,"{11, 12, 13, 14, 15, 16, 17, 18, 19}",2228,no,"[12, 12, 36]",39.799497,0.333333
9,Cam,"{11, 12, 13, 14, 15, 16, 17, 18, 19}",2235,yes,"[28, 20, 4]",34.641016,0.142857
